In [2]:
from Constants import *
from positions import loadpositions
from rhoova_func import *
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')


In [3]:
tasks=loadpositions()
tasks=tasks[:1]

In [5]:
tasks_df = pd.json_normalize(tasks, sep='_')
tasks_df['currency'] = tasks_df[['fixedRateBondDefinition_currency']].bfill(axis=1).iloc[:, 0]
tasks_df.drop(columns=['fixedRateBondDefinition_currency'], inplace=True)
columns=['trade_id', 'settlementDate', 'notional','currency', 'buySell', 'calculation_type','discountCurve']
tasks_df[columns]

,trade_id,settlementDate,notional,currency,buySell,calculation_type,discountCurve
0,FRB1001,2021-02-01,1000000,TRY,Buy,fixed_rate_bond,TRY_IRS


In [6]:
riskdata = {
  "id": "PORTFOLIO1",
  "name": "PORTFOLIO 1",
  "method": "VaR",
  "valuationDate": "2021-01-28",
  "valuationCurrency": "TRY",
  "riskMethod": "DELTANORMAL",
  "horizon": 252,
  "confidenceInterval": 0.99,
  "returnType": "NONE",
  "trend": False,
  "calendar": "Turkey",
  "timeBucket": ["0D","1D","1W","2W","3W","1M","2M","3M","6M","9M","1Y","2Y","3Y","4Y",
                "5Y","6Y","7Y","8Y","9Y","10Y","12Y","15Y","20Y","25Y","30Y","40Y","50Y"],
  "fillNa": "BACKWARD",
  "maxFillNaDays": 5,
  "tasks": tasks,
  "curves": [TRY_IRS,USD_IRS],
  "yieldData": yielddata.to_dict('records'),
  "marketData":marketdata.to_dict('records'),
  "volatilityData":voldata.to_dict('records'),
}

In [7]:
result=get_result(riskdata)

In [8]:
def sensitivities(data):
    bp=1/10000
    data['sensitivity'] =data['cashflow'] * (pow((1+data['rate']-bp),-1*data['timeToMatByYear'])-pow((1+data['rate']),-1*data['timeToMatByYear']))/bp
    return  data

In [9]:
def map_volatility(sensitivity,result):
    riskfactor_vol=pd.DataFrame(pd.DataFrame(result.get("deltaRiskFactors")).T.std())
    riskfactor_vol.columns=["volatility"]
    sensitivity=sensitivity.set_index("bin")
    tmp_df=pd.merge(sensitivity,riskfactor_vol,left_index=True, right_index=True)
    tmp_df["weighted_sensitivities"]=tmp_df["sensitivity"]*tmp_df["volatility"]*tmp_df["rate"]
    return tmp_df[["weighted_sensitivities"]]

In [10]:
regulatory_rw={"TRY_IRS0D":0.000065,"TRY_IRS1D":0.000065,"TRY_IRS1W":0.000065,"TRY_IRS2W":0.000065,
               "TRY_IRS3W":0.000065,"TRY_IRS1M":0.000065,"TRY_IRS2M":0.000082,"TRY_IRS3M":0.000097,
               "TRY_IRS6M":0.000079,"TRY_IRS9M":0.000079,"TRY_IRS1Y":0.000091,"TRY_IRS2Y":0.000103,
               "TRY_IRS3Y":0.000148,"TRY_IRS4Y":0.000248,"TRY_IRS5Y":0.000314,"TRY_IRS6Y":0.000425,
               "TRY_IRS7Y":0.000486,"TRY_IRS8Y":0.000565,"TRY_IRS9Y":0.000619,"TRY_IRS10Y":0.000612,
               "TRY_IRS12Y":0.000701,"TRY_IRS15Y":0.000795,"TRY_IRS20Y":0.000879,"TRY_IRS25Y":0.000922,
               "TRY_IRS30Y":0.000891,"TRY_IRS40Y":0.001186,"TRY_IRS50Y":0.001499}
rw=pd.DataFrame([regulatory_rw]).T
rw.columns=["RW"]
def map_rw(sensitivity,rw):
    sensitivity=sensitivity.set_index("bin")
    tmp_df=pd.merge(sensitivity,rw,left_index=True, right_index=True)
    tmp_df["weighted_sensitivities"]=tmp_df["sensitivity"]*tmp_df["RW"]
    return tmp_df[["weighted_sensitivities"]]

In [12]:
rf_bucket=pd.DataFrame(result.get("forDV01").get("TRY_IRS"))
rf_bucket

,bin,cashflow,rate,timeToMatByYear
0,TRY_IRS0D,0.000000,0.001208,0.000000
1,TRY_IRS1D,0.000000,0.001208,0.002778
2,TRY_IRS1W,0.000000,0.001208,0.019444
3,TRY_IRS2W,0.000000,0.001208,0.038889
4,TRY_IRS3W,0.000000,0.001208,0.058333
5,TRY_IRS1M,0.000000,0.001208,0.077778
6,TRY_IRS2M,0.000000,0.001585,0.163889
7,TRY_IRS3M,27403.155809,0.002114,0.252778
8,TRY_IRS6M,15343.495392,0.002275,0.505556
9,TRY_IRS9M,5357.247462,0.002693,0.758333


In [13]:
rf_sensitivities=sensitivities(rf_bucket)
rf_sensitivities

,bin,cashflow,rate,timeToMatByYear,sensitivity
0,TRY_IRS0D,0.000000,0.001208,0.000000,0.000000e+00
1,TRY_IRS1D,0.000000,0.001208,0.002778,0.000000e+00
2,TRY_IRS1W,0.000000,0.001208,0.019444,0.000000e+00
3,TRY_IRS2W,0.000000,0.001208,0.038889,0.000000e+00
4,TRY_IRS3W,0.000000,0.001208,0.058333,0.000000e+00
5,TRY_IRS1M,0.000000,0.001208,0.077778,0.000000e+00
6,TRY_IRS2M,0.000000,0.001585,0.163889,0.000000e+00
7,TRY_IRS3M,27403.155809,0.002114,0.252778,6.909035e+03
8,TRY_IRS6M,15343.495392,0.002275,0.505556,7.731080e+03
9,TRY_IRS9M,5357.247462,0.002693,0.758333,4.043771e+03


In [14]:
weighted_sensitivities =map_volatility(rf_sensitivities,result)
weighted_sensitivities

,weighted_sensitivities
TRY_IRS0D,0.000000
TRY_IRS1D,0.000000
TRY_IRS1W,0.000000
TRY_IRS2W,0.000000
TRY_IRS3W,0.000000
TRY_IRS1M,0.000000
TRY_IRS2M,0.000000
TRY_IRS3M,0.668943
TRY_IRS6M,0.606943
TRY_IRS9M,0.320443


In [15]:
reg_weighted_sensitivities=map_rw(rf_sensitivities,rw)
reg_weighted_sensitivities

,weighted_sensitivities
TRY_IRS0D,0.000000
TRY_IRS1D,0.000000
TRY_IRS1W,0.000000
TRY_IRS2W,0.000000
TRY_IRS3W,0.000000
TRY_IRS1M,0.000000
TRY_IRS2M,0.000000
TRY_IRS3M,0.670176
TRY_IRS6M,0.610755
TRY_IRS9M,0.319458


In [18]:
###################Our VaR Result##########################
confidenceinterval=0.99
rf_corr=pd.DataFrame(result.get("riskFactorsCorr"))
vol=np.sqrt(weighted_sensitivities.T@rf_corr@weighted_sensitivities)
vol.values[0][0]
#var=vol*norm.ppf(confidenceinterval) 
#var
#result.get("VaR")

2232.5714068004036

In [19]:
## Correlation Scenarios
Base_scenario = rf_corr.applymap(lambda x : x*1)
High_scenario = rf_corr.applymap(lambda x : min(x*1.25,1))
Low_scenario = rf_corr.applymap(lambda x : max(x*2 - 1 , 0.75 * x))
Base_scenario.style.set_caption('Intra-bucket correlation for Baseline scenario ')

,TRY_IRS0D,TRY_IRS1D,TRY_IRS1W,TRY_IRS2W,TRY_IRS3W,TRY_IRS1M,TRY_IRS2M,TRY_IRS3M,TRY_IRS6M,TRY_IRS9M,TRY_IRS1Y,TRY_IRS2Y,TRY_IRS3Y,TRY_IRS4Y,TRY_IRS5Y,TRY_IRS6Y,TRY_IRS7Y,TRY_IRS8Y,TRY_IRS9Y,TRY_IRS10Y,TRY_IRS12Y,TRY_IRS15Y,TRY_IRS20Y,TRY_IRS25Y,TRY_IRS30Y,TRY_IRS40Y,TRY_IRS50Y
TRY_IRS0D,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS1D,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS1W,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS2W,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS3W,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS1M,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.689344,0.560497,0.501562,0.539786,0.506176,0.322171,0.149658,0.228530,0.232707,0.179771,0.189689,0.191483,0.200411,0.251140,0.258076,0.235686,0.243605,0.230992,0.287190,0.264825,0.232283
TRY_IRS2M,0.689344,0.689344,0.689344,0.689344,0.689344,0.689344,1.000000,0.658903,0.547179,0.606880,0.586678,0.295963,0.195512,0.226366,0.214815,0.180249,0.190820,0.206949,0.219437,0.247206,0.244218,0.237915,0.256453,0.242894,0.287023,0.250185,0.213036
TRY_IRS3M,0.560497,0.560497,0.560497,0.560497,0.560497,0.560497,0.658903,1.000000,0.596041,0.629788,0.585409,0.270765,0.145139,0.134968,0.129732,0.112860,0.101354,0.123715,0.137250,0.148444,0.154648,0.151931,0.168146,0.164197,0.190183,0.167748,0.146368
TRY_IRS6M,0.501562,0.501562,0.501562,0.501562,0.501562,0.501562,0.547179,0.596041,1.000000,0.923218,0.728505,0.237070,0.159040,0.153668,0.181858,0.142592,0.149448,0.156436,0.177779,0.225642,0.196180,0.206702,0.227548,0.217545,0.267022,0.249488,0.222528
TRY_IRS9M,0.539786,0.539786,0.539786,0.539786,0.539786,0.539786,0.606880,0.629788,0.923218,1.000000,0.931330,0.234766,0.149650,0.151962,0.183458,0.148714,0.152167,0.165121,0.191688,0.231124,0.204081,0.227911,0.250376,0.239675,0.288408,0.261752,0.229461


In [20]:

"""
We have the weighted net sensitivity matrix(WSk) and intrabucket correlation matrix above 
now we move to bucket level capital charge for bucket for three correlation(rho) scenarios and store it in a array

"""
Kb_base = np.sqrt(weighted_sensitivities.T@Base_scenario@weighted_sensitivities)
Kb_low = np.sqrt(weighted_sensitivities.T@Low_scenario@weighted_sensitivities)
Kb_high = np.sqrt(weighted_sensitivities.T@High_scenario@weighted_sensitivities)
Kb_matrix = np.array([Kb_base, Kb_low, Kb_high])
Kb_matrix

array([[[2232.5714068 ]],

       [[2193.81064964]],

       [[2270.67060721]]])